<a href="https://colab.research.google.com/github/itta0607/WIP/blob/main/wip_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ セル①：Google Driveマウント & ライブラリインストール

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install mediapipe


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


✅ セル②：ライブラリ読み込み & パス設定

In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

train_video_path = Path("/content/drive/MyDrive/gym_videos_train")
test_video_path = Path("/content/drive/MyDrive/gym_videos_test")
train_csv_path = Path("/content/drive/MyDrive/gym_keypoints/train")
test_csv_path = Path("/content/drive/MyDrive/gym_keypoints/test")


✅ セル③：MediaPipe初期化 & 関数定義

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

def process_video(video_path, output_csv_path, label):
    cap = cv2.VideoCapture(str(video_path))
    all_landmarks = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            row = []
            for landmark in result.pose_landmarks.landmark:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            row.append(label)
            all_landmarks.append(row)

    cap.release()

    if all_landmarks:
        df = pd.DataFrame(all_landmarks)
        os.makedirs(output_csv_path.parent, exist_ok=True)
        df.to_csv(output_csv_path, index=False)
        print(f"✅ Saved: {output_csv_path}")
    else:
        print(f"⚠️ No pose detected: {video_path}")


✅ セル④：学習用動画のCSV変換

In [ ]:
for class_folder in train_video_path.iterdir():
    if class_folder.is_dir():
        label = class_folder.name.replace('_train', '').replace('_test', '')
        for video_file in class_folder.glob("*"):
            if video_file.suffix.lower() in [".mp4", ".mov", ".m4v"]:
                output_csv = train_csv_path / label / (video_file.stem + ".csv")
                process_video(video_file, output_csv, label)

✅ Saved: /content/drive/MyDrive/gym_keypoints/train/arm_curl/IMG_1795 (4).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/arm_curl/IMG_1795 (3).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/arm_curl/IMG_1795 (2).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/arm_curl/IMG_1795 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/arm_curl/IMG_1795.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bench_press/IMG_1792 (4).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bench_press/IMG_1792 (3).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bench_press/IMG_1792 (2).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bench_press/IMG_1792 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bench_press/IMG_1792.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bent_over_row/IMG_1794 (4).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/train/bent_over_row/IMG_1794 (3).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/t

✅ セル⑤：テスト用動画のCSV変換





In [ ]:
for class_folder in test_video_path.iterdir():
    if class_folder.is_dir():
        label = class_folder.name.replace('_train', '').replace('_test', '')
        for video_file in class_folder.glob("*"):
            if video_file.suffix.lower() in [".mp4", ".mov", ".m4v"]:
                output_csv = test_csv_path / label / (video_file.stem + ".csv")
                process_video(video_file, output_csv, label)


✅ Saved: /content/drive/MyDrive/gym_keypoints/test/arm_curl/IMG_1795 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/arm_curl/IMG_1795.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/arm_curl/IMG_2179 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/arm_curl/IMG_2179.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bench_press/IMG_1792 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bench_press/IMG_1792.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bench_press/IMG_2175 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bench_press/IMG_2175.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bent_over_row/IMG_1794 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bent_over_row/IMG_1794.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bent_over_row/IMG_2177 (1).csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/bent_over_row/IMG_2177.csv
✅ Saved: /content/drive/MyDrive/gym_keypoints/test/chest_press/IMG_1

 ✅ セル⑥：学習・テスト用CSV読み込み

In [ ]:
X_train, y_train = load_dataset(train_csv_path)
X_test, y_test = load_dataset(test_csv_path)


✅ セル⑦：データセット読み込み関数

In [ ]:
def load_dataset(csv_dir):
    data = []
    labels = []
    for label_folder in csv_dir.iterdir():
        if label_folder.is_dir():
            label = label_folder.name
            for csv_file in label_folder.glob("*.csv"):
                df = pd.read_csv(csv_file)

                # 数値列だけを取り出して平均をとる
                numeric_df = df.select_dtypes(include=[np.number])
                mean_row = numeric_df.mean().values

                data.append(mean_row)
                labels.append(label)
    return np.array(data), np.array(labels)


✅ セル⑧：分類器の学習と評価

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("\n📊 Classification Report:\n")
print(classification_report(y_test, y_pred))
print("✅ Accuracy:", accuracy_score(y_test, y_pred))



📊 Classification Report:

                precision    recall  f1-score   support

      arm_curl       0.75      0.75      0.75         4
   bench_press       0.75      0.75      0.75         4
 bent_over_row       0.50      1.00      0.67         4
   chest_press       1.00      0.50      0.67         4
  dumbbell_fly       0.75      0.75      0.75         4
   hammer_curl       1.00      0.50      0.67         4
 lat_pull_down       1.00      0.75      0.86         4
 lateral_raise       0.50      0.50      0.50         4
 leg_extension       1.00      0.50      0.67         4
     leg_press       1.00      1.00      1.00         4
       pec_fly       1.00      1.00      1.00         4
shoulder_press       0.67      1.00      0.80         4
         squat       0.80      1.00      0.89         4

      accuracy                           0.77        52
     macro avg       0.82      0.77      0.77        52
  weighted avg       0.82      0.77      0.77        52

✅ Accuracy: 0.7692

✅ セル⑨：CSVファイル数の確認（学習・テスト用）

In [ ]:
import glob

train_csv_files = glob.glob(str(train_csv_path / "*" / "*.csv"))
test_csv_files = glob.glob(str(test_csv_path / "*" / "*.csv"))

print("✅ Train CSV数:", len(train_csv_files))
print("✅ Test CSV数:", len(test_csv_files))


✅ Train CSV数: 65
✅ Test CSV数: 52
